In [33]:
type Result<'T> =
    | Success of 'T
    | Error of string

type LogState = { Logs: string list }

type ResultLog<'T> = Result<'T> * LogState

type ResultBuilder() =
    member _.Bind(x: ResultLog<'T>, f) = 
        match x with
        | Success v, (state: LogState) -> 
            let (result, newState) = f v
            match result with
            | Success _ ->
              (result, { state with Logs = state.Logs @ newState.Logs })
            | Error e -> (Error e, { state with Logs = state.Logs @ newState.Logs @ [sprintf "Error: %s" e] })
        | Error e, state ->
            (Error e, state)
            
    member _.Return(x) = 
        printfn "Return: %A" x
        (Success x, { Logs = [] })
    
    member _.Zero(x) = 
        printfn "Zero: %A" x
        (Success x, { Logs = [] })

    member _.Yield(x) =
        printfn "Yield: %A" x
        (Success x, { Logs = [] })

    member _.For(x: ResultLog<'T>, f) = 
        match x with
        | Success v, (state: LogState) -> 
            let (result, newState) = f v
            match result with
            | Success _ ->
              (result, { state with Logs = state.Logs @ newState.Logs })
            | Error e -> (Error e, { state with Logs = state.Logs @ newState.Logs @ [sprintf "Error: %s" e] })
        | Error e, state ->
            (Error e, state)

    [<CustomOperation("log")>]
    member _.Log (x, message: string): ResultLog<'T> = 
        match x with
        | Success v, (state: LogState) -> 
            (Success v, { state with Logs = state.Logs @ [message] })
        | Error e, state ->
            (Error e, state)

let result = ResultBuilder()

let multiply10 x = 
    if x > 0 then (Success (x * 10), { Logs = [sprintf "multiply10 successful: %d" (x * 10)] })
    else (Error "x must be positive", { Logs = ["multiply10 failed: x must be positive"] })
    
let add1 x = 
    if x < 100 then (Success (x + 1), { Logs = [sprintf "add1 successful: %d" (x + 1)] })
    else (Error "result too large", { Logs = ["add1 failed: x too large"] })

let num1 = result {
    let! a = multiply10 5
    let! b = add1 a
    return b
}

match num1 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs -> 
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs

let num2 = result {
    log "starting calculation" // no error, properly sets the first message of the Logs
    let! a = multiply10 50
    // log "starting calculation" // This expression was expected to have type 'int' but here has type 'unit'  
    let! b = add1 a
    let! c = multiply10 b
    return c
}

match num2 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs


Return: 51
Run: (Success 51, { Logs = ["multiply10 successful: 50"; "add1 successful: 51"] })
Final result: 51
Logs: ["multiply10 successful: 50"; "add1 successful: 51"]
Yield: ()
Run: (Error "result too large",
 { Logs =
    ["starting calculation"; "multiply10 successful: 500";
     "add1 failed: x too large"; "Error: result too large";
     "Error: result too large"] })
An error occurred: result too large
Logs: ["starting calculation"; "multiply10 successful: 500";
 "add1 failed: x too large"; "Error: result too large";
 "Error: result too large"]


In [37]:
type Result<'T> =
    | Success of 'T
    | Error of string

type LogState = { Logs: string list }

type ResultLog<'T> = Result<'T> * LogState

type ResultWriterBuilder() =
    member _.Bind(x: ResultLog<'T>, f) = 
        match x with
        | Success v, (state: LogState) -> 
            let (result, newState) = f v
            match result with
            | Success _ ->
              (result, { state with Logs = state.Logs @ newState.Logs })
            | Error e -> (Error e, { state with Logs = state.Logs @ newState.Logs @ [sprintf "Error: %s" e] })
        | Error e, state ->
            (Error e, state)
            
    member _.Return(x) = 
        printfn "Return: %A" x
        (Success x, { Logs = [] })

let resultWriter = ResultWriterBuilder()

let multiply10 x = 
    if x > 0 then (Success (x * 10), { Logs = [sprintf "multiply10 successful: %d" (x * 10)] })
    else (Error "x must be positive", { Logs = ["multiply10 failed: x must be positive"] })
    
let add1 x = 
    if x < 100 then (Success (x + 1), { Logs = [sprintf "add1 successful: %d" (x + 1)] })
    else (Error "result too large", { Logs = ["add1 failed: x too large"] })

let num1 = resultWriter {
    let! a = multiply10 5
    let! b = add1 a
    return b
}

match num1 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs -> 
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs

let num2 = resultWriter {
    let! a = multiply10 50
    let! b = add1 a
    let! c = multiply10 b
    return c
}

match num2 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs

Return: 51
Final result: 51
Logs: ["multiply10 successful: 50"; "add1 successful: 51"]
An error occurred: result too large
Logs: ["multiply10 successful: 500"; "add1 failed: x too large";
 "Error: result too large"]


In [55]:
// Define types

type Result<'T, 'E> =
    | Success of 'T
    | Error of 'E

type LogState = { Logs: string list }

type ResultLog<'T> = Result<'T> * LogState

// Define computation expression

type ResultWriterBuilder() =
    member _.Bind((result, state: LogState), f) =
        match result with
        | Success value ->
            let (newResult, newState) = f value
            (newResult, { state with Logs = state.Logs @ newState.Logs })
        | Error e ->
            (Error e, state)

    member _.Return(value) =
        (Success value, { Logs = [] })


let resultWriter = ResultWriterBuilder()

// Define functions

let multiply10 x = 
    if x > 0 then (Success (x * 10), { Logs = [sprintf "multiply10 successful: %d" (x * 10)] })
    else (Error "x must be positive", { Logs = ["multiply10 failed: x must be positive"] })
    
let add1 x = 
    if x < 100 then (Success (x + 1), { Logs = [sprintf "add1 successful: %d" (x + 1)] })
    else (Error "result too large", { Logs = ["add1 failed: x too large"] })

// Run computation expressions

let num1 = resultWriter {
    let! a = multiply10 5
    let! b = add1 a
    return b
}

match num1 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs -> 
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs
    
// Final result: 51
// Logs: ["multiply10 successful: 50"; "add1 successful: 51"]

let num2 = resultWriter {
    let! a = multiply10 50
    let! b = add1 a
    let! c = multiply10 b
    return c
}

match num2 with
| Success finalResult, logs -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" logs.Logs
| Error e, logs ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" logs.Logs
    
// An error occurred: result too large
// Logs: ["multiply10 successful: 500"; "add1 failed: x too large"]

Final result: 51
Logs: ["multiply10 successful: 50"; "add1 successful: 51"]
An error occurred: result too large
Logs: ["multiply10 successful: 500"; "add1 failed: x too large"]


In [136]:
// Define types

type ResultWriter<'T, 'E, 'L> = Result<'T, 'E> * 'L list

// Define computation expression

type ResultWriterBuilder() =
    member _.Bind((result, log), f) =
        match result with
        | Success value ->
            let (newResult, newLog) = f value
            (newResult, log @ newLog)
        | Error e ->
            (Error e, log)

    member _.Return(value) =
        (Success value, [])


let resultWriter = ResultWriterBuilder()

// Define functions

let multiply10 x = 
    if x > 0 then
      let product = x * 10
      (Success product, [sprintf "multiply10 successful: %d" product])
    else (Error "x must be positive", ["multiply10 failed: x must be positive"])
    
let add1 x = 
    if x < 100 then
      let sum = x + 1
      (Success sum, [sprintf "add1 successful: %d" sum])
    else (Error "result too large", ["add1 failed: x too large"])

let log message = ResultWriter(Success (), [message])

// Define bind operator

let bind (result, log) f =
    match result with
    | Success value ->
        let (newResult, newLog) = f value
        (newResult, log @ newLog)
    | Error e ->
        (Error e, log)

let (>>=) x f = bind x f

let logBind message value = 
    (Success value, [message])

// Run computation expressions

let num1 = resultWriter {
    do! log "starting calculation 1"
    let! a = multiply10 5
    do! log "starting calculation 2"
    let! b = add1 a
    return b
}

match num1 with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" log
| Error e, log -> 
    printfn "An error occurred: %s" e
    printfn "Logs: %A" log
    
// Final result: 51
// Log: ["multiply10 successful: 50"; "add1 successful: 51"]

let num2 = resultWriter {
    do! log "starting calculation 1"
    let! a = multiply10 50
    do! log "starting calculation 2"
    let! b = add1 a
    let! c = multiply10 b
    return c
}

match num2 with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Log: %A" log
| Error e, log ->
    printfn "An error occurred: %s" e
    printfn "Log: %A" log
    
// An error occurred: result too large
// Log: ["multiply10 successful: 500"; "add1 failed: x too large"]

let num1BindInfix =
    (Success 50, [])
    >>= logBind "starting calculation 1"
    >>= multiply10
    >>= logBind "starting calculation 2"
    >>= add1

match num1BindInfix with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" log
| Error e, log ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" log

let num1BindPipe =
    bind (Success 5, ["starting calculation 1"]) multiply10
    |> (fun result -> bind result (logBind "starting calculation 2"))
    |> (fun result -> bind result add1)


match num1BindPipe with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" log
| Error e, log ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" log

let num1Bind2 =
    let initial = (Success 5, ["starting calculation 1"])
    let afterMultiply10 = bind initial multiply10
    let afterLog = bind afterMultiply10 (logInfix "starting calculation 2")
    let finalResult = bind afterLog add1
    finalResult

match num1Bind with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" log
| Error e, log ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" log

let num1Bind1 =
    bind (bind (bind (Success 5, ["starting calculation 1"]) multiply10) (logInfix "starting calculation 2")) add1

match num1Bind1 with
| Success finalResult, log -> 
    printfn "Final result: %d" finalResult
    printfn "Logs: %A" log
| Error e, log ->
    printfn "An error occurred: %s" e
    printfn "Logs: %A" log


Final result: 51
Logs: ["starting calculation 1"; "multiply10 successful: 50"; "starting calculation 2";
 "add1 successful: 51"]
An error occurred: result too large
Log: ["starting calculation 1"; "multiply10 successful: 500";
 "starting calculation 2"; "add1 failed: x too large"]
An error occurred: result too large
Logs: ["starting calculation 1"; "multiply10 successful: 500";
 "starting calculation 2"; "add1 failed: x too large"]
Final result: 51
Logs: ["starting calculation 1"; "multiply10 successful: 50"; "starting calculation 2";
 "add1 successful: 51"]
Final result: 51
Logs: ["starting calculation 1"; "multiply10 successful: 50"; "starting calculation 2";
 "add1 successful: 51"]
Final result: 51
Logs: ["starting calculation 1"; "multiply10 successful: 50"; "starting calculation 2";
 "add1 successful: 51"]


In [146]:

#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Abstractions"
#r "nuget: Microsoft.Extensions.Logging.Console"

open System
open Microsoft.Extensions.Logging

// Define the result type for computation
type Result<'T, 'E> = 
    | Success of 'T 
    | Error of 'E

// Define computation expression builder
type ResultWriterBuilder<'T, 'E>(logger: ILogger) =
    member _.Bind(result: Result<'T, 'E>, f: 'T -> Result<'T, 'E>) =
        match result with
        | Success value ->
            logger.LogInformation("Operation successful with value: {Value}", value)
            f value
        | Error e ->
            logger.LogError("Operation failed with error: {Error}", e)
            Error e

    member _.Return(value: 'T) =
        logger.LogInformation("Returning value: {Value}", value)
        Success value

// Sample functions that use the ILogger
let multiply10 (logger: ILogger) x = 
    if x > 0 then
        let product = x * 10
        logger.LogInformation("multiply10 successful: {Product}", product)
        Success product
    else
        logger.LogError("multiply10 failed: x must be positive")
        Error "x must be positive"
    
let add1 (logger: ILogger) x = 
    if x < 100 then
        let sum = x + 1
        logger.LogInformation("add1 successful: {Sum}", sum)
        Success sum
    else
        logger.LogError("add1 failed: x too large")
        Error "result too large"

// Instantiate the logger (normally done via dependency injection)
let loggerFactory = LoggerFactory.Create(fun builder ->
    builder.AddConsole() |> ignore)
let logger = loggerFactory.CreateLogger("MyLogger")

// Instantiate the computation expression builder
let resultWriter = ResultWriterBuilder<int, string>(logger)

// Run computation expressions
let runComputation() =
    let num1 = resultWriter {
        let! a = multiply10 logger 5
        let! b = add1 logger a
        return b
    }

    match num1 with
    | Success finalResult -> 
        printfn "Final result: %d" finalResult
    | Error e -> 
        printfn "An error occurred: %s" e

    let num2 = resultWriter {
        let! a = multiply10 logger 50
        let! b = add1 logger a
        let! c = multiply10 logger b
        return c
    }

    match num2 with
    | Success finalResult -> 
        printfn "Final result: %d" finalResult
    | Error e ->
        printfn "An error occurred: %s" e

runComputation()


Installed Packages Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Abstractions, 6.0.0 Microsoft.Extensions.Logging.Console, 6.0.0

info: MyLogger[0]
      multiply10 successful: 50
info: MyLogger[0]
      Operation successful with value: 50
info: MyLogger[0]
      add1 successful: 51
info: MyLogger[0]
      Operation successful with value: 51
Final result: 51
info: MyLogger[0]
      Returning value: 51
info: MyLogger[0]
      multiply10 successful: 500
info: MyLogger[0]
      Operation successful with value: 500
fail: MyLogger[0]
      add1 failed: x too large
fail: MyLogger[0]
      Operation failed with error: result too large
An error occurred: result too large


In [150]:
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Abstractions"
#r "nuget: Microsoft.Extensions.Logging.Console"

open System
open Microsoft.Extensions.Logging

// Define the result type for computation
type Result<'T> = 
    | Success of 'T 
    | Error of string

// Define computation expression builder
type ResultWriterBuilder(logger: ILogger) =
    member _.Bind(result: Result<int>, f: int -> Result<int>) =
        match result with
        | Success value ->
            logger.LogInformation("Operation successful with value: {Value}", value)
            f value
        | Error e ->
            logger.LogError("Operation failed with error: {Error}", e)
            result

    member _.Return(value: int) =
        logger.LogInformation("Returning value: {Value}", value)
        Success value

    member _.Logger = logger

// Modify functions to operate without an explicit logger parameter
let multiply10 x = 
    if x > 0 then
        let product = x * 10
        Success product
    else
        Error "x must be positive"
    
let add1 x = 
    if x < 100 then
        let sum = x + 1
        Success sum
    else
        Error "result too large"

// Instantiate the logger (normally done via dependency injection)
let loggerFactory = LoggerFactory.Create(fun builder ->
    builder.AddConsole() |> ignore)
let logger = loggerFactory.CreateLogger("MyLogger")

// Instantiate the computation expression builder
let resultWriter = ResultWriterBuilder(logger)

// Run computation expressions
let runComputation() =
    let num1 = resultWriter {
        resultWriter.Logger.LogInformation("starting calculation 1")
        let! a = multiply10 5
        resultWriter.Logger.LogInformation("starting calculation 2")
        let! b = add1 a
        return b
    }

    match num1 with
    | Success finalResult -> 
        printfn "Final result: %d" finalResult
    | Error e -> 
        printfn "An error occurred: %s" e

    let num2 = resultWriter {
        let! a = multiply10 50
        let! b = add1 a
        let! c = multiply10 b
        return c
    }

    match num2 with
    | Success finalResult -> 
        printfn "Final result: %d" finalResult
    | Error e ->
        printfn "An error occurred: %s" e

runComputation()


Installed Packages Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Abstractions, 6.0.0 Microsoft.Extensions.Logging.Console, 6.0.0

info: ResultWriter Logger[0]
      starting calculation 1
info: ResultWriter Logger[0]
      Operation successful with value: 50
info: ResultWriter Logger[0]
      starting calculation 2
info: ResultWriter Logger[0]
      Operation successful with value: 51
info: ResultWriter Logger[0]
      Returning value: 51
Final result: 51
info: ResultWriter Logger[0]
      Operation successful with value: 500
An error occurred: result too large
fail: ResultWriter Logger[0]
      Operation failed with error: result too large
